In [1]:
from __future__ import division
import os, warnings, tensorflow as tf
from utils import *

In [2]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

## Information

We load pre-trained models to verify the accuracy figures.  
Training code will be released at a later date.

## Load datasets

In [3]:
from dsl.fashionmnist_dsl import FashionMNISTDSL

### Load confidential dataset $\mathcal{D}_C$

In [4]:
train_dsl = FashionMNISTDSL(batch_size = 100, mode='train', normalize_channels=False, shuffle_each_epoch=False, seed=666)
val_dsl   = FashionMNISTDSL(batch_size = 100, mode='val', normalize_channels=False, shuffle_each_epoch=False, seed=666)
test_dsl  = FashionMNISTDSL(batch_size = 100, mode='test', normalize_channels=False, shuffle_each_epoch=False, seed=666)

Loading train data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
Loading val data
datasets/fashionmnist/train-images-idx3-ubyte
(60000, 28, 28, 1)
(60000,)
Loading test data
datasets/fashionmnist/t10k-images-idx3-ubyte
(10000, 28, 28, 1)
(10000,)


### Obtain dataset parameters

In [5]:
width, height, channels = train_dsl.get_sample_shape()

is_multilabel = train_dsl.is_multilabel()
num_classes = train_dsl.get_num_classes()
num_batches = train_dsl.get_num_batches()

### Load attacker datasets $\mathcal{D}_A$

Syn-Uniform Retraining (Tramer _et al._ [1])

In [6]:
from dsl.uniform_dsl import UniformDSL

sample_shape = (width, height, channels)
uniform_dsl = UniformDSL(batch_size=100, mode='train', shape=sample_shape, sample_limit=1000, seed=666)

ImageNet samples

In [7]:
from dsl.imagenet_dsl import ImagenetDSL

resize = (width, height)
img_noise_train_dsl = ImagenetDSL(batch_size=100, mode='train', resize=resize, normalize_channels=True, num_to_keep=1000, start_batch=1, end_batch=1, seed=666)

Loading train data
datasets/Imagenet64/train_data_batch_1.npy


## Create models

In [8]:
import tensorflow as tf
from models.deepcnn import DeepCNN
from models.vae import NewHSVNVAE
from models.defended_model import DefenseModel

TensorFlow version 1.14.0 has been patched using tfdeterminism version 0.3.0
TensorFlow version 1.14.0 has been patched using tfdeterminism version 0.3.0


In [9]:
tf.reset_default_graph()

with tf.compat.v1.variable_scope("true_model", reuse=tf.AUTO_REUSE):
    true_model = DeepCNN(
        batch_size=100,
        height=height,
        width=width,
        channels=channels,
        num_classes=num_classes,
        multilabel=is_multilabel,
        is_training=False,
        var_prefix='true_model'
    )

with tf.compat.v1.variable_scope("copy_model"):
    copy_model = DeepCNN(
        batch_size=100,
        height=height,
        width=width,
        channels=channels,
        num_classes=num_classes,
        multilabel=is_multilabel,
        is_training=True,
        var_prefix='copy_model',
        learning_rate=0.001
    )

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Adam Learning rate: 0.001


In [10]:
undefended_saver = tf.train.Saver()

In [11]:
from cfg import cfg, config

In [12]:
with tf.variable_scope("vae", reuse=tf.AUTO_REUSE):
    vae = NewHSVNVAE(
            batch_size=cfg.batch_size,
            height=height, width=width,
            channels=channels,
            num_classes=2,
            is_training=False,
            z_size=32,
            random_draws=1,
            noise_mean=5.0
         )
    
    defended_model = DefenseModel(
        true_model,
        vae,
        threshold=[0.1, 0.1],
        defender_type="vae_svm",
        use_recon_input=False
    )
    
    oracle_model = defended_model




Tensor("vae/Decoder/conv2d_transpose/LeakyRelu:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("vae/Decoder/conv2d_transpose_1/LeakyRelu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("vae/Decoder/conv2d_transpose_2/LeakyRelu:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("vae/Decoder/conv2d_transpose_3/LeakyRelu:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("vae/Decoder/conv2d_transpose_4/Sigmoid:0", shape=(?, 28, 28, 1), dtype=float32)

Tensor("vae/Decoder_1/conv2d_transpose/LeakyRelu:0", shape=(?, 2, 2, 256), dtype=float32)
Tensor("vae/Decoder_1/conv2d_transpose_1/LeakyRelu:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("vae/Decoder_1/conv2d_transpose_2/LeakyRelu:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("vae/Decoder_1/conv2d_transpose_3/LeakyRelu:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("vae/Decoder_1/conv2d_transpose_4/Sigmoid:0", shape=(?, 28, 28, 1), dtype=float32)


In [13]:
defended_saver = tf.train.Saver()

In [14]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

## Accuracy of models

In this section, we show the accuracy of defended vs. undefended models on pretrained models for various attackers.

### Syn extracted model

In [15]:
logdir_copy = os.path.join('modeldir/fashionmnist-syn-undefended/')
undefended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-syn-undefended/model_epoch_49


In [16]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 25.33%


#### VarDefend-Defended $S_D$ ($10,000$ budget)

In [17]:
logdir_copy = os.path.join('modeldir/fashionmnist-syn-defended/')
defended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-syn-defended/model_epoch_49


In [18]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 7.56%


### AdvPD Extracted Models

#### Undefended (100K Budget)

In [19]:
logdir_copy = os.path.join('modeldir/fashionmnist-advpd-undefended/')
undefended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-advpd-undefended/model_epoch_99


In [20]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 81.15%


#### VarDetect-Defended

In [21]:
logdir_copy = os.path.join('modeldir/fashionmnist-advpd-defended/')
defended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-advpd-defended/model_epoch_99


In [22]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 75.87%


### AdvPD Extracted Models

#### Undefended (100K Budget)

In [23]:
logdir_copy = os.path.join('modeldir/fashionmnist-npd-undefended/')
undefended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-npd-undefended/model_step_4438


In [24]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 80.77%


#### Defended

In [25]:
logdir_copy = os.path.join('modeldir/fashionmnist-npd-defended/')
defended_saver.restore(sess, tf.train.latest_checkpoint(logdir_copy))

INFO:tensorflow:Restoring parameters from modeldir/fashionmnist-npd-defended/model_step_4


In [26]:
compute_agreement_and_accuracy(true_model, copy_model, sess, test_dsl)

Test Set Accuracy: 12.24%
